In [1]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm
from itertools import combinations, permutations
import time

In [2]:
data_A = np.loadtxt('../data/TSPA.csv', delimiter=';').astype(np.int64)
data_B = np.loadtxt('../data/TSPB.csv', delimiter=';').astype(np.int64)
data_C = np.loadtxt('../data/TSPC.csv', delimiter=';').astype(np.int64)
data_D = np.loadtxt('../data/TSPD.csv', delimiter=';').astype(np.int64)

In [3]:
def create_cost_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2) + cost.reshape(1, -1)).round().astype(np.int64)

def create_dist_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    #cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2)).round().astype(np.int64)

In [4]:
cost_matrix_A = create_cost_matrix(data_A)

In [5]:
cost_matrix_A

array([[  84, 2032, 2098, ..., 4159, 3783, 1514],
       [1633,  483, 2398, ..., 3349, 2266,  817],
       [ 720, 1419, 1462, ..., 3640, 3149,  964],
       ...,
       [2782, 2371, 3641, ..., 1461, 2908, 2554],
       [2558, 1440, 3302, ..., 3060, 1309, 1773],
       [1234,  936, 2062, ..., 3651, 2718,  364]])

In [6]:
cost_matrix_B = create_cost_matrix(data_B)

In [7]:
cost_matrix_C = create_cost_matrix(data_C)

In [8]:
cost_matrix_D = create_cost_matrix(data_D)

In [9]:
dist_matrix_A = create_dist_matrix(data_A)
dist_matrix_B = create_dist_matrix(data_B)
dist_matrix_C = create_dist_matrix(data_C)
dist_matrix_D = create_dist_matrix(data_D)

In [10]:
def plot(data, solution):
    data_ordered = np.array([data[i] for i in solution])
    all_data = np.array([data[i] for i in range(200)])

    plt.figure(figsize=(10, 10), dpi=80)

    plt.scatter(data_ordered[:,0], data_ordered[:,1], s=data_ordered[:,2]/data_ordered[:,2].max()*200, c='b')
    plt.scatter(all_data[:,0], all_data[:,1], s=all_data[:,2]/all_data[:,2].max()*200, c='b')
    plt.plot(data_ordered[:,0], data_ordered[:,1], 'y-')
    plt.plot([data_ordered[0,0], data_ordered[-1,0]], [data_ordered[0,1], data_ordered[-1,1]], 'y-')
    plt.show()

In [11]:
def calculate_performance(cycle, cost_matrix):
    total_sum = 0
    for i in range(len(cycle)-1):
        total_sum += cost_matrix[cycle[i], cycle[i+1]]
    total_sum += cost_matrix[cycle[-1], cycle[0]]
    return total_sum

In [12]:
np.arange(0, 100)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [13]:
@jit()
def random_solution(cost_matrix, limit=100):
    random_solution_list = np.arange(0, 100)
    np.random.shuffle(random_solution_list)
    return random_solution_list[:limit]

solution = random_solution(cost_matrix_A, 100)
print(solution)

[92 71 84 48  0 87 35 66 89  3 85 26 40 74 55 17 34 90 10 47 39  4 23 60
 58 88 54 42 22 68 67 98 33 36 99  2 27 76 44 83 11 63 75 61 72 96 31 32
 29 30 62 37 94 97 24 93 46 70 41 25  7 77  1 16 82 51 21 53  6 52 14 69
 57 81 28 95 73 80 19 20 59 79 56 13 43  9 65 18 38 78 15 12  8 64 86 50
 45  5 49 91]


In [14]:
# %%timeit
# random_solution(cost_matrix_A)

In [15]:
@jit()
def greedy_cycle(cost_matrix, current_id, limit=100):
    all_ids = set(list(range(0,len(cost_matrix))))
    all_ids.remove(current_id)
    solution = [current_id]
    
    for _ in range(1):
        min_val = 99999
        min_id = -1
        for next_id in all_ids:
            if cost_matrix[current_id][next_id] < min_val:
                min_val = cost_matrix[current_id][next_id]
                min_id = next_id
        solution.append(min_id)
        all_ids.remove(min_id)
        current_id = min_id
    
    while len(solution) < limit:
        min_delta = 99999
        min_id = -1
        insert_id = -1
        for i in range(len(solution)-1):
            for next_id in all_ids:
                delta = cost_matrix[solution[i]][next_id] + cost_matrix[next_id][solution[i+1]] - cost_matrix[solution[i]][solution[i+1]]
                if delta < min_delta:
                    min_delta = delta
                    min_id = next_id
                    insert_id = i
        for next_id in all_ids:
            delta = cost_matrix[solution[-1]][next_id] + cost_matrix[next_id][solution[0]] - cost_matrix[solution[-1]][solution[0]]
            if delta < min_delta:
                min_delta = delta
                min_id = next_id
                insert_id = i
        solution.insert(insert_id+1, min_id)
        all_ids.remove(min_id)

    return np.array(solution)

In [16]:
calculate_performance(greedy_cycle(cost_matrix_A, 2), cost_matrix_A)

77328

In [17]:
# %%timeit
# greedy_cycle(cost_matrix_A, 0)

In [18]:
@jit()
def calculate_closest_neighbours(cost_matrix, n_neighbours=10):
    def extract_nodes(ll):
        return [x[1] for x in ll]
    
    diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
    np.fill_diagonal(diag_matrix, np.inf)
    
    new_cost_matrix = cost_matrix + diag_matrix

    closest_neighbours = dict()
    for i in range(len(new_cost_matrix)):
        closest_neighbours[i] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
    return closest_neighbours

In [19]:
jit(nopython=True)
def calculate_closest_neighbours_solution(cost_matrix, solution, n_neighbours=10):
    def extract_nodes(ll):
        return [solution[x[1]] for x in ll]
    
    diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
    np.fill_diagonal(diag_matrix, np.inf)
    
    new_cost_matrix = cost_matrix[solution, :][:, solution] + diag_matrix[solution, :][:, solution]

    closest_neighbours = dict()
    for i in range(len(new_cost_matrix)):
        closest_neighbours[solution[i]] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
    return closest_neighbours

In [20]:
pre = calculate_closest_neighbours_solution(cost_matrix_A, list(range(100)))

In [21]:
solution = list(range(15)) + list(range(100, 115))

In [22]:
# %%timeit
# calculate_closest_neighbours_solution(cost_matrix_A, solution)

In [23]:
def calculate_change_before(enhanced_solution, cost_matrix, position_1, position_2):
    return (cost_matrix[enhanced_solution[position_1], enhanced_solution[position_2]] + cost_matrix[enhanced_solution[position_1-1], enhanced_solution[position_2-1]] -
            ( cost_matrix[enhanced_solution[position_1-1], enhanced_solution[position_1]] + cost_matrix[enhanced_solution[position_2-1], enhanced_solution[position_2]] )
            )

def calculate_change_after(enhanced_solution, cost_matrix, position_1, position_2):
    return (cost_matrix[enhanced_solution[position_1], enhanced_solution[position_2]] + cost_matrix[enhanced_solution[(position_1+1)%len(enhanced_solution)], enhanced_solution[(position_2+1)%len(enhanced_solution)]] -
            ( cost_matrix[enhanced_solution[position_1], enhanced_solution[(position_1+1)%len(enhanced_solution)]] + cost_matrix[enhanced_solution[position_2], enhanced_solution[(position_2+1)%len(enhanced_solution)]] )
            )

In [24]:
import time

In [29]:
# @jit()
def enhance_solution(initial_solution, closest_neighbours, cost_matrix):

    enhanced_solution = list(initial_solution)
    previous_solution = 0
    current_solution = calculate_performance(initial_solution, cost_matrix)
    total_time = 0
    while True:
        previous_solution = current_solution
        best_delta = 0                  # e.g. -10
        best_edges_positions = None     # e.g. [1, 10]
        best_type = None                # before or after
        neighbours_positions = {node: enhanced_solution.index(node) for node in enhanced_solution}
        for position in range(len(initial_solution)):
            for neighbour in closest_neighbours[enhanced_solution[position]]:
                start = time.time()
                position_neighbour = neighbours_positions[neighbour]
                end = time.time()
                total_time += end - start 
                if abs(position - position_neighbour) == 1: continue
                    
                
                for swap_type in ('before', 'after'):
                    
                    if swap_type == 'before':
                        delta = calculate_change_before(enhanced_solution, cost_matrix, position_1=position, position_2=position_neighbour)
                        
                    elif swap_type == 'after':
                        delta = calculate_change_after(enhanced_solution, cost_matrix, position_1=position, position_2=position_neighbour)
                    
                    if delta < best_delta:
                        best_delta = delta
                        best_edges_positions = [position, position_neighbour]
                        best_type = swap_type
        
                
                

        
                        
        
        # break
        if best_delta == 0:
            return enhanced_solution, total_time
        else:
            if best_type == 'before':
                if best_edges_positions[0] < best_edges_positions[1]:
                    enhanced_solution = enhanced_solution[:best_edges_positions[0]] + enhanced_solution[best_edges_positions[0]:best_edges_positions[1]][::-1] + enhanced_solution[best_edges_positions[1]:]
                else:
                    enhanced_solution = enhanced_solution[best_edges_positions[1]:best_edges_positions[0]] + list(enhanced_solution[best_edges_positions[0]:] + enhanced_solution[:best_edges_positions[1]])[::-1]
            elif best_type == 'after':
                if best_edges_positions[0] < best_edges_positions[1]:
                    enhanced_solution = enhanced_solution[:(best_edges_positions[0]+1)%len(enhanced_solution)] + enhanced_solution[(best_edges_positions[0]+1)%len(enhanced_solution):(best_edges_positions[1]+1)][::-1] + enhanced_solution[(best_edges_positions[1]+1):]
                else:
                    enhanced_solution = enhanced_solution[best_edges_positions[1]+1:best_edges_positions[0]+1] + list(enhanced_solution[(best_edges_positions[0]+1):] + enhanced_solution[:best_edges_positions[1]+1])[::-1]
                    

            
                
        current_solution = calculate_performance(enhanced_solution, cost_matrix)
        assert current_solution < previous_solution
        
    return 0, 1


In [30]:
from tqdm import tqdm

In [31]:
def start_enhancement(starting_solution_type: str, cost_matrix, distance_matrix):
    total_costs = []
    total_cost_random = 0
    total_time = 0
    for i in tqdm(range(len(distance_matrix))):
        # print(i)
        if starting_solution_type == 'greedy':
            initial_solution = greedy_cycle(cost_matrix, i, limit=100)
        elif starting_solution_type == 'random':
            initial_solution = random_solution(cost_matrix, limit=100)
        else:
            return 'Bad type'
        initial_cost = calculate_performance(initial_solution, cost_matrix)
        total_cost_random += initial_cost

        closest_neighbours = calculate_closest_neighbours_solution(distance_matrix, initial_solution, n_neighbours=10)
        enhanced_solution, time_spent = enhance_solution(initial_solution, closest_neighbours, distance_matrix)
        total_costs.append(calculate_performance(enhanced_solution, cost_matrix))
        total_time += time_spent
    total_costs = np.array(total_costs)
    return (total_costs.mean(), total_costs.min(), total_costs.max()) #, total_cost_random / len(cost_matrix), total_time, total_time / len(cost_matrix)
        
        

        


In [32]:
start_enhancement('random', cost_matrix=cost_matrix_C, distance_matrix=dist_matrix_C)

100%|██████████| 200/200 [00:41<00:00,  4.84it/s]


(73774.225, 72647, 74991)

In [50]:
start_enhancement('random', cost_matrix=cost_matrix_D, distance_matrix=dist_matrix_D)

100%|██████████| 200/200 [00:55<00:00,  3.61it/s]


(75083.995, 218860.99, 0.04975008964538574, 0.0002487504482269287)